# Reacting to answers with a validation function

First set up the notebook

In [7]:
#load "load.fsx"

open Informedica.Utils.Lib.BCL
open Informedica.OpenAI.Lib
open Ollama.Operators

## Define a validation function

The function validates the answer and returns a Result which can either be Ok or Error. The Error response is fed back to the LLM to retry getting the right answer.

In [8]:
let validator =
    fun s -> 
    match s |> String.contains "3" with
    | true -> "1 + 1 could be 3" |> Ok
    | false -> "the answer should include the possibility of a '1 + 1 could be 3'" |> Error

## A conversation without the validation function

In [9]:
let conversation =
    "You are a helpful asistent on general topics"
    |> init Ollama.Models.``openchat:7b``
    >>? """
    How does the saying go like the whole is more than the individual parts?
    """
    |> fun conv -> 
        conv |> Conversation.print
        conv

Starting conversation with openchat:7b

Options:
{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_penalty":null,"frequency_penalty":null,"mirostat":0,"mirostat_tau":null,"mirostat_eta":null,"penalize_newline":null,"stop":[],"numa":null,"num_ctx":2048,"num_batch":null,"num_gqa":null,"num_gpu":null,"main_gpu":null,"low_vram":null,"f16_kv":null,"vocab_only":null,"use_mmap":null,"use_mlock":null,"rope_frequency_base":null,"rope_frequency_scale":null,"num_thread":null}

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"messages":[{"content":"You are a helpful asistent on general topics","role":"system"},{"content":"\n    How does the saying go like the whole is more than the individual parts?\n    ","role":"user"}],"model":"openchat:7b","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_la

## The conversation contiued with the validation function

The conversation continues, but now with the validation function.

In [10]:
conversation
>>! ("""
That would mean 1 + 1 could be ?
""" |> Message.userWithValidator validator)
|> Conversation.print

ℹ INFO: 
EndPoint: http://localhost:11434/api/chat
Payload:
{"messages":[{"content":"You are a helpful asistent on general topics","role":"system"},{"content":"\n    How does the saying go like the whole is more than the individual parts?\n    ","role":"user"},{"content":" The saying you're referring to is \"The whole is greater than the sum of its parts.\" This phrase means that the overall value, quality, or function of a system, group, or entity is greater than the combined values, qualities, or functions of its individual components. It emphasizes the importance of understanding and appreciating the interconnectedness and synergy within a whole, rather than just focusing on the individual parts.","role":"assistant"},{"content":"\nThat would mean 1 + 1 could be ?\n","role":"user"}],"model":"openchat:7b","options":{"num_keep":null,"seed":101,"num_predict":null,"top_k":null,"top_p":null,"tfs_z":null,"typical_p":null,"repeat_last_n":64,"temperature":0.0,"repeat_penalty":null,"presence_